Jonathan Oleson

Bi-CGSTAB Project - Building test matrices

MA 5631, Advanced Numerical Linear Algebra, Dr. Allan Struthers

Michigan Technological University

In [1]:
using LinearAlgebra, SparseArrays, MatrixMarket
#using Plots

Define a finite difference discretization for the Poisson equation on a 150X150 grid of unknowns

In [6]:
# do we need to divide by h (or maybe 2h)?
n = 10 # use 150 for the test problem
m = n^2
A = spzeros(m,m)
for j in 1:n
    for i in 1:n
        k = i + (j-1)*n
        A[k,k] = -4.0
        #print(i,' ',j,' ',k,'\n') #troubleshooting
        if (1<=(i+1) & (i+1)<=n); A[k,k+1] = 1.0; end
        if (1<=(i-1) & (i-1)<=n); A[k,k-1] = 1.0; end
        if (1<=(j+1) & (j+1)<=n); A[k,k+n] = 1.0; end
        if (1<=(j-1) & (j-1)<=n); A[k,k-n] = 1.0; end
    end
end

eigen(Matrix(A)).values

100-element Vector{Float64}:
 -7.837971894457994
 -7.601493012891357
 -7.601493012891345
 -7.365014131324731
 -7.228707415119568
 -7.228707415119567
 -6.992228533552936
 -6.992228533552922
 -6.749815973232769
 -6.749815973232763
 -6.619442935781142
 -6.51333709166614
 -6.513337091666136
  ⋮
 -1.4866629083338634
 -1.3805570642188603
 -1.2501840267672328
 -1.250184026767232
 -1.0077714664470698
 -1.0077714664470674
 -0.771292584880436
 -0.7712925848804346
 -0.6349858686752753
 -0.39850698710864363
 -0.39850698710864324
 -0.16202810554201086

Export $A$ to a .mtx file.

In [10]:
if n == 150; mmwrite("test_problem_1.mtx",A); else; print("A is not the right size"); end

A is not the right size

In [4]:
# test stuff to make sure I know how to use the Julia cholesky function
testmat = rand(n,n)
testmat = testmat*testmat'
#cholesky(testmat)
eigen(testmat).values

10-element Vector{Float64}:
  0.003746600467795444
  0.1322042254408411
  0.20967264076641065
  0.43634529418046286
  0.703445004195287
  0.922780816112448
  1.1820716841147583
  1.4393142787595232
  2.2203717115510337
 25.353721383369113

The paper preconditions the problem as $L^{-1}.A.U^{-1}.\tilde x = L^{-1}.b$. Create one of the preconditioner matrices for A using a Cholesky factorization (paper calls for modified incomplete, but use plain for now). The other one is just the transpose of the first.

In [3]:
eigen(Matrix(A)).values # negative eigenvalues? cholesky won't work since A isn't SPD!

100-element Vector{Float64}:
 -7.837971894457992
 -7.601493012891357
 -7.601493012891353
 -7.365014131324727
 -7.228707415119571
 -7.228707415119569
 -6.992228533552937
 -6.992228533552932
 -6.749815973232768
 -6.749815973232758
 -6.619442935781139
 -6.5133370916661395
 -6.513337091666136
  ⋮
 -1.4866629083338625
 -1.380557064218859
 -1.2501840267672322
 -1.2501840267672315
 -1.0077714664470685
 -1.007771466447066
 -0.7712925848804354
 -0.7712925848804345
 -0.6349858686752753
 -0.398506987108643
 -0.3985069871086423
 -0.16202810554201033

In [4]:
cholesky(Hermitian(A))

LoadError: PosDefException: matrix is not positive definite; Cholesky factorization failed.